# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████████████████████████████████████████████████████████████████████| 5/5 [00:59<00:00, 11.96s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Mainstays Handheld 3-in-1 Fan / Flashlight / Power Bank for $6 + free shipping w/ $35\nDetails: That's a great price for a multipurpose tool and perfect for travel. Choose pickup or spend $35 to avoid the $6.99 shipping charge. Buy Now at Walmart\nFeatures: foldable USB rechargeable Model: MFP2406B\nURL: https://www.dealnews.com/products/Mainstays/Handheld-3-in-1-Fan-Flashlight-Power-Bank/492299.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Apple iPhone 16 Plus 128GB for Mint Mobile: 50% off 2-Year Unlimited Plan w/ purchase
Details: Get the Apple iPhone 16 Plus, along with two years of Mint's Unlimited plan, for $54 a month. That's half off the Unlimited plan, down from $720 to $360. It requires port-in, and an upfront payment for 24 months of service. The plan renews at its normal price (currently $3

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The 100Ah 12.8V LiFePO4 Rechargeable Battery is a reliable power solution ideal for various applications like off-grid solar power systems, RVs, and electric vehicles. This battery is designed for longevity with up to 6000 cycles and offers efficient performance even in extreme temperatures. Lightweight and compact, it enables easy installation and management. Safety features include built-in protection against overcharging and short circuits, ensuring a secure power source for your needs.', price=106.0, url='https://www.dealnews.com/100-Ah-12-8-V-Li-Fe-PO4-Rechargeable-Battery-for-106-free-shipping/21756793.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The Apple iPhone 16 Plus is a powerful smartphone featuring a 128GB storage capacity, perfect for storing all your apps, photos, and videos. It has a sleek design and is equipped with impressive technical specifications that deliver outstanding performance. The device supports advanced features like unlimited talk and text, high-speed data access, and a free mobile hotspot feature, making it a versatile choice for users who demand connectivity and performance.', price=360.0, url='https://www.dealnews.com/Apple-iPhone-16-Plus-128-GB-for-Mint-Mobile-50-off-2-Year-Unlimited-Plan-w-purchase/21756933.html?iref=rss-c142'), Deal(product_description="The TCL QM5K Series 50-inch 4K HDR QD-Mini LED UHD Smart TV boasts a stunning 3840x2160 resolution, providing crystal clear visuals for all your entertainment needs. This model supports HDR 10+ for enhanced color and contrast and is compatible with Google TV, allowing voice control through Google Home